<a href="https://colab.research.google.com/github/kjh1311990/face/blob/main/Fuzzy_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install komm

     |████████████████████████████████| 51kB 2.5MB/s 
  Created wheel for komm: filename=komm-0.7.1-cp36-none-any.whl size=56607 sha256=bc1a4c0a6d72546f482ea6e2449f913076c280665dd94d10f7ee94b731750f0f
  Stored in directory: /root/.cache/pip/wheels/22/1e/97/4a65e9a6e202aa360d36ec6a822b31cea491e2ec55788cf71f
Successfully built komm


In [55]:
import komm
import numpy as np
code = komm.BCHCode(11,365)

In [56]:
import numpy as np

PARAMS_M = 2047
PARAMS_N = 512

PARAMS_LOGQ = 10
PARAMS_LOGL = 6

PARAMS_Q = 1 << PARAMS_LOGQ
PARAMS_L = 1 << PARAMS_LOGL

In [57]:
def EMBLEM_Encode(m, shift):
    encm = ((m << 1) + 1) << (shift - 1);
    return encm.reshape(PARAMS_M, 1);

In [58]:
def EMBLEM_Decode(m, shift):
    decm = m >> shift;
    return decm.reshape(PARAMS_M, 1);

In [59]:
A = np.random.randint(low = -PARAMS_L/2, high = PARAMS_L/2+1, size = PARAMS_N * PARAMS_M, dtype = int)
A.resize(PARAMS_M, PARAMS_N)

In [60]:
b = np.random.randint(low = 0, high = PARAMS_Q, size = PARAMS_N, dtype = int);
b.resize(PARAMS_N,1);

In [15]:
m = np.random.randint(low = 0, high = 2, size = 133, dtype = int);

In [16]:
encm = code.encode(m);
encm.reshape(2047,1);

In [132]:
c = np.remainder(np.matmul(A,b) + EMBLEM_Encode(encm, PARAMS_LOGQ-1), PARAMS_Q);

In [26]:
b1 = np.random.randint(low = 0, high = 2, size = PARAMS_N, dtype = int);
b1.resize(PARAMS_N,1);

b2 = np.random.randint(low = 0, high = 2, size = PARAMS_N, dtype = int);
b2.resize(PARAMS_N,1);

bprime = b;# + b1 - b2;

In [133]:
cprime = np.remainder(c - np.matmul(A,bprime), PARAMS_Q)

In [134]:
decm = EMBLEM_Decode(cprime, PARAMS_LOGQ-1).reshape(2047)

In [135]:
mprime = code.decode(decm)

In [136]:
np.sum(np.abs(m - mprime))

0

In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
from keras.models import load_model
from keras.models import Model

model = load_model('/content/drive/MyDrive/Notebooks/weights.h5')
model2 = Model(model.input, model.layers[-2].output)

In [100]:
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

img = np.zeros((5,224,224,3));
feature = np.zeros((5,512));
for i in range(0,5):
  img[i] = image.load_img('/content/drive/MyDrive/Notebooks/face/output/1/'+ str(i) +'.jpg', target_size=(224, 224))
  x = image.img_to_array(img[i])
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  feature[i] = model2.predict(x)

In [101]:
t = feature.max(axis=0) - feature.min(axis=0);

In [128]:
for i in range(0,5):
  for j in range(0,512):
    if t[j] > 1:
      feature[i][j] = (feature[i][j] / (4* t[j]))

In [129]:
feature = feature.astype(int)

In [130]:
b = feature[0]
bprime = feature[1]

In [131]:
b = b.reshape(PARAMS_N,1);
bprime = bprime.reshape(PARAMS_N,1);